## N-Gram

在N-Gram模型中，通过将文本分割成连续的N个词的组合，来近似地描述词序列的联合概率。假设一个词出现的概率仅依赖于它前面的N-1个词。换句话说，利用有限的上下文信息(N-1)词来近似地预测下一个词的概率。

### 创建一个Bigram字符预测模型
整体结构为：

<img src="./images/BiGram.png" alt="img" style="zoom: 60%;" />

> 1.构建实验语料库

In [4]:
# 构建一个玩具数据集
corpus = [ "我喜欢吃苹果",
        "我喜欢吃香蕉",
        "她喜欢吃葡萄",
        "他不喜欢吃香蕉",
        "他喜欢吃苹果",
        "她喜欢吃草莓"]
corpus

['我喜欢吃苹果', '我喜欢吃香蕉', '她喜欢吃葡萄', '他不喜欢吃香蕉', '他喜欢吃苹果', '她喜欢吃草莓']

> 2.把句子分成N个"Gram"(分词)

In [5]:
#  定义一个分词函数，将文本转换为单个字符的列表
def tokenize(text):
    return [char for char in text]
print("单字列表")
for text in corpus:
    tokens = tokenize(text)
    print(tokens)

单字列表
['我', '喜', '欢', '吃', '苹', '果']
['我', '喜', '欢', '吃', '香', '蕉']
['她', '喜', '欢', '吃', '葡', '萄']
['他', '不', '喜', '欢', '吃', '香', '蕉']
['他', '喜', '欢', '吃', '苹', '果']
['她', '喜', '欢', '吃', '草', '莓']


> 3.计算每个Bigram在语料库中的词频

In [37]:
# 一些测试
l = [1,2,3,4,5]
l[0:1]
l[0:-1]
t = tuple(l[0:-1])
print(t)
prefix = t[:-1]
prefix
sum(l)

(1, 2, 3, 4)


15

In [39]:
# 定义计算N-Gram词频的函数
from collections import defaultdict, Counter
# defaultdict(Counter) value为Counter类型
# Counter是一个无序的容器类型，以字典的键值对形式存储，其中元素作为key，其计数作为value
def count_ngrams(corpos, n):
    ngrams_count = defaultdict(Counter) # 创建一个字典，存储N-Gram计数
    for text in corpos: # 遍历语料库中的每个文本
        tokens = tokenize(text) # 对文本进行分词
        for i in range(len(tokens) - n + 1): # 遍历分词结果，根据n值生成N个元素
            ngram = tuple(tokens[i:i+n]) # 创建一个N-Gram元祖
            prefix = ngram[:-1] # 前缀(n - 1)
            token = ngram[-1] # 获取N-Gram的目标单字
            ngrams_count[prefix][token] += 1 # 更新 N-Gram计数
    return ngrams_count
bigram_counts = count_ngrams(corpus,2) # 计算 bigram 词频
print("bigram 词频:")
for prefix, counts in bigram_counts.items():
    print("{}: {}".format("".join(prefix),dict(counts)))


bigram 词频:
我: {'喜': 2}
喜: {'欢': 6}
欢: {'吃': 6}
吃: {'苹': 2, '香': 2, '葡': 1, '草': 1}
苹: {'果': 2}
香: {'蕉': 2}
她: {'喜': 2}
葡: {'萄': 1}
他: {'不': 1, '喜': 1}
不: {'喜': 1}
草: {'莓': 1}


> 4.定义计算 N-Gram 出现概率的函数

In [54]:
def ngram_prebabilities(ngram_counts):
    ngram_probs = defaultdict(Counter) # 存储N-Gram出现的概率
    for prefix, tokens_counts in ngram_counts.items():
        total_count = sum(tokens_counts.values()) # 计算当前前缀下共有多少个N-Grams
        for token, count in tokens_counts.items(): # 遍历每个prefix
            ngram_probs[prefix][token] = count / total_count # 计算每个N-Gram出现的概率
    return ngram_probs
bigram_probs = ngram_prebabilities(bigram_counts)
print("bigram出现的概率统计:")
for prefix, probs in bigram_probs.items():
    print("{}: {}".format("".join(prefix),dict(probs)))

bigram出现的概率统计:
我: {'喜': 1.0}
喜: {'欢': 1.0}
欢: {'吃': 1.0}
吃: {'苹': 0.3333333333333333, '香': 0.3333333333333333, '葡': 0.16666666666666666, '草': 0.16666666666666666}
苹: {'果': 1.0}
香: {'蕉': 1.0}
她: {'喜': 1.0}
葡: {'萄': 1.0}
他: {'不': 0.5, '喜': 0.5}
不: {'喜': 1.0}
草: {'莓': 1.0}


> 5.根据Bigram的概率统计，定义生成下一个词的函数

In [64]:
# 测试下max函数
d1 = {'a':1,'b':2,'c':3}
print(max(d1))
c1 = Counter('dsfadsfdfffff')
print(c1)
max(c1, key=c1.get) # Count与dict有区别，如果指定key，输出counter中key最大的值而不是value
print(range(3))
for i in range(len(d1)):
    print(i)

c
Counter({'f': 7, 'd': 3, 's': 2, 'a': 1})
range(0, 3)
0
1
2


In [68]:
def gen_next_token(prefix, bigram_probs):
    if not prefix in bigram_probs:
        return None # 如果prefix不在dict中，返回none
    next_token_probs = bigram_probs[prefix]
    next_token = max(next_token_probs, key=next_token_probs.get) # 返回出现概率最高的词汇
    return next_token

> 6.定义生成连续文本的函数

In [66]:
def gen_text(prefix, ngram_probs, n, length=6):
    tokens = list(prefix)
    for _ in range(length - len(prefix)):
        next_token = gen_next_token(tuple(tokens[-(n-1):]), ngram_probs)
        if not next_token:
            break
        tokens.append(next_token)
    return "".join(tokens)

In [72]:
# 给定一个prefix，输出完整的句子
prefix = '我'
n = 2
print("\ntext:", gen_text(prefix, bigram_probs, 2))


text: 我喜欢吃苹果
